In [1]:
# From:  https://quirkyai.wordpress.com/2017/06/07/text-generation-in-lines-with-lstm-rnn-in-python-tensorflow/

In [ ]:
import numpy as np, tflearn, sys, tensorflow as tf
from tensorflow.contrib.keras.python.keras.utils import np_utils
input = open("sherlock.txt").read().lower()
chars = sorted(list(set(input)))
charint = dict((char,ints) for ints, char in enumerate(chars))
intchar = dict((ints,char) for ints, char in enumerate(chars))
filename = 'sherlockLSTM_model'
seqlen = 100
lstmhid = 320
keeprate = 0.80
train = []
true = []
tf.reset_default_graph()
for i in range(0, len(input)-seqlen, 1):
    train.append([charint[char] for char in input[i:i+seqlen]])
    true.append(charint[input[i+seqlen]])
X = np.reshape(train, (len(train), seqlen, 1))/float(len(chars))
y = np_utils.to_categorical(true)
net = tflearn.input_data(shape=(None, X.shape[1], X.shape[2]))
net = tflearn.lstm(net, lstmhid)
net = tflearn.fully_connected(net, y.shape[1], activation='softmax')
net = tflearn.regression(net, optimizer='adam', learning_rate=0.005, loss ='categorical_crossentropy')
model = tflearn.DNN(net, checkpoint_path = 'charmodel/model.tfl.ckpt')
model.fit(X, y, snapshot_epoch=True,
snapshot_step=5000, n_epoch=20, batch_size = 128)
model.save(filename)

Training Step: 2100  | total loss: 2.62971 | time: 1184.251s
| Adam | epoch: 001 | loss: 2.62971 -- iter: 268800/561733


In [ ]:
for _ in range(5):
    p = train[np.random.randint(0,len(train)-1)]
    print("Seed:")
    print("\"",''.join([intchar[value] for value in p]), "\"")
    for _ in range(100):
        sys.stdout.write((intchar[np.argmax(model.predict((np.reshape(p, (1, len(p), 1))/float(len(chars)))))]))
        p.append(np.argmax(model.predict((np.reshape(p, (1, len(p), 1))/float(len(chars))))))
        p = p[1:len(p)]
        print("\n============================\n")